In [443]:
import numpy as np
import open3d as o3d
import pathlib
import math
import itertools

## Loading in Point cloud data

Problems encountered: Trouble cleaning with scans with a depth perception problem, might need to investigate order of code execution

Todo: Remove more of the floor manually before applying DBSCAN

Trouble with: 
P003, 
P010 (ground isnt cleaned??) 
P011 (weird capture?) 
P013 (ground isnt cleaned??)
P014 (ground isnt cleaned??)
P015 (ground)
P017


P005 / P006 have some spurious points around its edges, especially around the feet.

In [450]:
# Load ply file
path = "./data/P003 2022-01-25 01_15_26.ply"

if pathlib.Path(path).exists():
    pcd = o3d.io.read_point_cloud(path)
    print('PLY file loaded')
else:
    raise FileNotFoundError("File not found.")

print('Shape of points', np.asarray(pcd.points).shape)
print('Shape of colors', np.asarray(pcd.colors).shape)

PLY file loaded
Shape of points (465839, 3)
Shape of colors (465839, 3)


In [411]:
o3d.visualization.draw_geometries([pcd])

In [415]:
import matplotlib.pyplot as plt

def viewClustersViaColours(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Test function to visualise the clusters by colours
    
    Experiment with parameters of cluster_dbscan() for potentially better results..
    
    eps: distance between neighbours in a cluster
    min_points: minimum number of points required to form a cluster
    
    Note: MUST reload in data after finishing visual.
    '''
    
    labels = np.array(pcd.cluster_dbscan(eps=0.03, min_points=3, print_progress=True))
    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcd])

viewClustersViaColours(pcd)

point cloud has 223 clusters


# Preprocessing

In [451]:
# Perform plane segmentation
plane_model, inliers = pcd.segment_plane(distance_threshold=0.03,
                                          ransac_n=3,
                                          num_iterations=200)
# Here, distance_threshold is the maximum distance a point can be from the plane to be considered an inlier,
# ransac_n is the number of initial points to sample,
# num_iterations is the number of iterations the algorithm performs.

# inliers contains the indices of the points that belong to the plane
# We can use this to create a new point cloud containing only these points:
plane = pcd.select_by_index(inliers)
plane.paint_uniform_color([1,0,0])

# We can also create a point cloud containing everything else:
pcd = pcd.select_by_index(inliers, invert=True)

# Visualize
o3d.visualization.draw_geometries([pcd, plane])


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [452]:
def minimalBoundingBox(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Applies a minimal bounding box, isolating the human subject from the scene.
    This helps to remove majority of the unnecessary points.
    
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
    
    Returns:
        type: o3d.cpu.pybind.geometry.PointCloud'     
    '''
    
    points = np.asarray(pcd.points)
    
    min_x, min_y, min_z = points.min(axis=0)[:3]
    max_x, max_y, max_z = points.max(axis=0)[:3]

    padding = 0.5 # need to calculate, cant be preset value.
    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(padding*min_x, padding*min_y ,min_z), max_bound=(padding*max_x, padding*max_y, max_z))
    bbox.get_minimal_oriented_bounding_box()
    bbox.color = [1,0,0]

    # Select points within the bounding box
    pcd_in_bbox = pcd.crop(bbox)
    
    o3d.visualization.draw_geometries([pcd_in_bbox])
    
    return pcd_in_bbox

## TODO! - REVIEW CODE / REFACTOR
def bounding_box_minor(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Cleans the floor??
    
    Brain no work 
    
    '''
    
    # Create bounding box:
    bounds = [[-math.inf, math.inf], [-math.inf, math.inf], [0, 0.0458]]  # set the bounds
    
    bounding_box_points = list(itertools.product(*bounds))  # create limit points
    bounding_box = o3d.geometry.AxisAlignedBoundingBox.create_from_points(
        o3d.utility.Vector3dVector(bounding_box_points))  # create bounding box object
    
    _, inliers = pcd.segment_plane(distance_threshold=0.01,
                                          ransac_n=3,
                                          num_iterations=1000)
    plane = pcd.select_by_index(inliers)
    pcd = pcd.select_by_index(inliers, invert=True)
    
    # Crop the point cloud using the bounding box:
    pcd_croped = pcd.crop(bounding_box)
    dists = np.asarray(pcd.compute_point_cloud_distance(pcd_croped))
    indices = np.where(dists > 0.001)[0]
    pcd_cropped_inv = pcd.select_by_index(indices)

    # Display the cropped point cloud:
    # self.visualise(pcd_croped, windowName="pcd cropped")
    # o3d.visualization.draw_geometries([pcd_cropped_inv])
        
    return pcd_cropped_inv


def downsample_clean(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Performs downsampling, outlier removal and bounding box removal
    
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
    
    Returns:
        type: None 
    '''

    # voxel downsampling: reducing overall num of pts
    voxel_size = 0.02
    downsampled = pcd.voxel_down_sample(voxel_size) 
    
    # outlier cleaning
    _, ind = pcd.remove_statistical_outlier(nb_neighbors=100, std_ratio=2.0)
    cleaned_pcd = pcd.select_by_index(ind)
    
    # minimal bounding box
    pcd = minimalBoundingBox(cleaned_pcd)
    pcd = bounding_box_minor(pcd)

    # paint removal parts
    epsilon = 0.01
    points = np.asarray(pcd.points)
    indices = np.where(np.abs(points[:, 2] < epsilon))[0]
    
    pcd_in_color = pcd.select_by_index(indices)
    pcd_in_color.paint_uniform_color([1,1,0])
    pcd = pcd.select_by_index(indices, invert=True)
    
    o3d.visualization.draw_geometries([pcd])

    return pcd

pcd = downsample_clean(pcd)

In [447]:
from collections import Counter

def isolateLargestCluster(pcd: o3d.cpu.pybind.geometry.PointCloud, labels: np.ndarray) -> None:
    '''
    Uses DBSCAN to group and isolate the largest point cloud 
       
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
        labels (np.ndarray): Array of clusters identified by labels
    
    Returns:
        type: o3d.cpu.pybind.geometry.PointCloud
    '''

    # finding the label of largest cluster + ignoring noisy points labeled -1
    counts = Counter(labels)
    largest_cluster_label = max(counts.items(), key=lambda x: x[1] if x[0] != -1 else -1)[0]

    # get all pts / colors of largest cluster
    largest_cluster_points = np.array(pcd.points)[labels == largest_cluster_label]
    largest_cluster_colors = np.array(pcd.colors)[labels == largest_cluster_label]
    
    # new point cloud from the largest cluster pts w/ colors
    largest_cluster_pcd = o3d.geometry.PointCloud()
    largest_cluster_pcd.points = o3d.utility.Vector3dVector(largest_cluster_points)
    largest_cluster_pcd.colors = o3d.utility.Vector3dVector(largest_cluster_colors)
    
    return largest_cluster_pcd
    
labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=3, print_progress=True))
pcd = isolateLargestCluster(pcd, labels)
o3d.visualization.draw_geometries([pcd])